In [3]:
import os
import glob
import contextily as ctx
from asp_plot.icesat2 import ICESat2

In [5]:
directory = "/Users/ben/Dropbox/UW_Shean/WV/2022/WV03_20220417_1040010074793300_1040010075633C00"

dem_fn = glob.glob(os.path.join(directory, "stereo*/*DEM_1m.tif"))[0]

geojson_fn = "/Users/ben/Dropbox/UW_Shean/WV/2022/utqiagvik_wv_2022_sliderule_poly.geojson"

# atl06_fn = "/Users/ben/Dropbox/UW_Shean/ICESat/icesat2_atl06_sr_10mRes_utqiagvik_WorldCover_water_crop_MayJune_allTime.parquet"

icesat_directory = os.path.join(directory, "icesat_data")
os.makedirs(icesat_directory, exist_ok=True)

plots_directory = os.path.join(directory, "asp_plots")
os.makedirs(plots_directory, exist_ok=True)

map_crs = "EPSG:32604"

ctx_kwargs = {
    "crs": map_crs,
    "source": ctx.providers.Esri.WorldImagery,
    "attribution_size": 0,
    "alpha": 0.5,
}

In [7]:
icesat2 = ICESat2(dem_fn=dem_fn, geojson_fn=geojson_fn)

In [8]:
atl06 = icesat2.pull_atl06_data()
atl06.head()


ICESat-2 ATL06 request processing



,rms_misfit,segment_id,pflags,spot,gt,y_atc,x_atc,rgt,cycle,region,h_sigma,h_mean,w_surface_window_final,dh_fit_dx,n_fit_photons,geometry,esa-worldcover-.time,esa-worldcover-.file_id,esa-worldcover-.flags,esa-worldcover-.value
time,,,,,,,,,,,,,,,,,,,,
2018-10-19 01:22:42.849838080,0.057618,396539,0,6,10,2075.261719,7947945.5,312,1,3,0.033456,0.712761,3.000000,0.004047,5,POINT (-156.33771 71.24194),1.309046e+12,493921239040,0,90.0
2018-10-19 01:22:42.852651008,0.059513,396540,0,6,10,2075.311279,7947965.5,312,1,3,0.019844,0.823991,3.000000,0.009106,10,POINT (-156.33778 71.24212),1.309046e+12,493921239040,0,90.0
2018-10-19 01:22:42.855464704,0.093339,396541,0,6,10,2075.361572,7947985.5,312,1,3,0.024544,0.905593,112.929657,-0.000860,15,POINT (-156.33786 71.2423),1.309046e+12,493921239040,0,90.0
2018-10-19 01:22:42.858280448,0.091363,396542,0,6,10,2075.408936,7948005.5,312,1,3,0.021550,0.905599,3.000000,0.002180,18,POINT (-156.33793 71.24247),1.309046e+12,493921239040,0,90.0
2018-10-19 01:22:42.861097984,0.104050,396543,0,6,10,2075.448486,7948025.5,312,1,3,0.024073,0.946535,3.000000,0.001642,19,POINT (-156.338 71.24265),1.309046e+12,493921239040,0,90.0
